In [ ]:
!pip install pyshp fiona

     |████████████████████████████████| 219 kB 5.0 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37324 sha256=a75a1a29bf1e01179b34fb5f290c159a369aec5f609c165c084bcc67a351d5e4
  Stored in directory: /root/.cache/pip/wheels/43/f8/87/53c8cd41545ba20e536ea29a8fcb5431b5f477ca50d5dffbbe
Successfully built pyshp


In [ ]:
import numpy as np
import pandas as pd
import datetime
import math
import shapefile
from shapely import geometry
import fiona
from collections import defaultdict
from google.colab import drive
from datetime import date, timedelta

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'sustainlab'
assert FOLDERNAME is not None, "[!] Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\ Drive/$FOLDERNAME/

Mounted at /content/drive
/content/drive/My Drive/sustainlab


In [ ]:
# Import the Earth Engine Python Package
import ee

# Initialize the Earth Engine object, using the authentication credentials.
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=wsAPe7nG-UIvP7zXwOzWxU_mNjHZGhlkBAkiwRa4BsQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh4aBK2fe66rlixAfSJQFPM3a-OtqxDFBmJAhNFDL1h-AUFAFBHF7g

Successfully saved authorization token.


In [ ]:
sentinel = ee.ImageCollection("COPERNICUS/S2").select(['B4', 'B3', 'B2'])
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").select(['SR_B4', 'SR_B3', 'SR_B2'])

In [ ]:
fname = 'hold'
device = 'sentinel'
with fiona.open(f'shapefiles/{fname}.shp') as shapefile:
    # geoms = [feature['geometry'] for feature in shapefile]
    # ids = [feature['properties']['id'] for feature in shapefile]
    # dates = [feature['properties']['date'] for feature in shapefile]
    i = 0
    for j in range(2900*0, len(shapefile)):
        feature = shapefile[j]
        i += 1
        geom = feature['geometry']
        geom = geom['coordinates'][0][:4]
        xmin = geom[0][0]
        ymin = geom[0][1]
        xmax = geom[2][0]
        ymax = geom[2][1]
        geom = ee.Geometry.Rectangle([ymin, xmin, ymax, xmax])
        # print([ymin, xmin, ymax, xmax])
        id = feature['properties']['id']
        capturedate = date.fromisoformat(feature['properties']['date'])
        delta = timedelta(days=+35)
        startdate = str(capturedate - delta)
        enddate = str(capturedate + delta)
        # print(id, capturedate)
        # print(startdate, enddate)
        img = None
        if device == 'sentinel':
            s2 = sentinel.filter(ee.Filter.date(startdate, enddate)).filterBounds(geom).sort('CLOUDY_PIXEL_PERCENTAGE').first()
            img = s2.visualize(min=[0.0, 0.0, 0.0], max=[5000, 5000, 5000])
        else:
            s2 = landsat.filter(ee.Filter.date(startdate, enddate)).filterBounds(geom).sort('CLOUD_COVER_LAND').first()
            img = s2.visualize(min=[0.0, 0.0, 0.0], max=[30000.0, 30000.0, 30000.0])
        config = {
            'folder':f'xview{device}{fname}',
            'scale': 10,
            'region': geom
        }
        task = ee.batch.Export.image.toDrive(img, id, **config)
        task.start()
        if i%100 == 0:
            print(i)
        if i == 2900:
            print('start next from j=',j,'+1')
            break
    print(i, 'done')

100
200
300
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1866 done


In [ ]:
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(10)
else:
  print(task.status())

{'state': 'UNSUBMITTED'}


In [ ]:
!earthengine task list

Instructions for updating:
non-resource variables are not supported in the long term
Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, and then retry.


In [ ]:
!earthengine task cancel all

Instructions for updating:
non-resource variables are not supported in the long term
Canceling task "G4FJX6WLIRQAG3DUO3RMNNH3"
Canceling task "ZH77677KHEI77UPYGAWLJPLI"
Canceling task "YS3RC674AQNYZ2U5BLPY36A5"
Canceling task "MIURFSO5PXX7GMVIFEPF63TF"
Canceling task "DGMHAJIZKL3N23IUSAPR7NNM"
Canceling task "ERQLTJL7PQIHIR5MTRUZM4UJ"
Canceling task "DQHNZ3JZXQQ5B27HNVQBTP4H"
Canceling task "L4N2WAIIEPP26ARCVZJIRLMU"
Canceling task "4RQNFLJY6UNUYO5QRRIG3RSV"
Canceling task "GMJPSQJW6D542WNMHTEUWFRE"
Canceling task "QYXDEI7MMIQJDTDLQUXJVUDI"
Canceling task "KWQPFWAOLDCNLY6QYRSDRJLK"
Canceling task "27XIOVWIX6Z7JD4MVGZSIJYG"
Canceling task "ZFRC36NYS3I4WURXJCFY2BPD"
Canceling task "TLTL2U7CLSB3JJUXNUKYHWG5"
Canceling task "6AKTO2MAOUTD6AIO7AWW5TRS"
Canceling task "AB3C7XHF5SOI5WVXVTIJXT4F"
Canceling task "GALZNLNCRYXPQNMSEGGNL2DG"
Canceling task "LAIAVOWSNGNOLEZ7SY4OVNLO"
Canceling task "CM4C7IZ7O3DG2UERZDM3KQ7L"
Canceling task "ITAVHVEKJW4FSG4RULXNHPO6"
Canceling task "QEEMLW3BXR7NRPLWH